# Mapping

[Mappings](https://nexus-forge.readthedocs.io/en/latest/interaction.html#mapping) are pre-defined and declarative rules that encode the logic on how to transform a specific data source into [Resources](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) that eventually conform to targeted schemas supported by the configured [Model](https://nexus-forge.readthedocs.io/en/latest/interaction.html#modeling). 

This notebook specifically demonstrates the `DictionaryMapping` wich is based on a JSON structure that represent a targeted structure along with Python code that will apply desired transformations on the data source.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource

In [4]:
from kgforge.specializations.mappings import DictionaryMapping

## Data

In [5]:
scientists = [
    {
        "id": 123,
        "name": "Marie Curie",
        "gender": "female",
        "middle_name": "Salomea",
    },
    {
        "id": 456,
        "name": "Albert Einstein",
        "gender": "male",
        "middle_name": "(missing)",
    },
]

## Mapping data to a targeted template

### basics

Note: DemoModel and RdfModel schemas have not been synchronized yet. The following cell is to be run with DemoModel.

In [6]:
forge.types()

Managed entity types:
   - Activity
   - Contribution
   - Dataset
   - Entity
   - Ontology
   - Person


In [6]:
forge.template("Contribution")

{
    id: ""
    agent:
    {
        id: ""
        type: Agent
    }
    hadPlan:
    {
        id: ""
        type: Plan
        label: ""
    }
    hadRole:
    {
        id: ""
        type: Role
        label: ""
    }
}


In [7]:
mapping_simple = DictionaryMapping("""
    type: Contribution
    agent:
    {
        type: Person
        name: x.name
    }
""")

In [8]:
resources_simple = forge.map(scientists, mapping_simple)

In [9]:
print(*resources_simple, sep="\n")

{
    type: Contribution
    agent:
    {
        type: Person
        name: Marie Curie
    }
}
{
    type: Contribution
    agent:
    {
        type: Person
        name: Albert Einstein
    }
}


### missing values

In [10]:
mapping_na = DictionaryMapping("""
    type: Contribution
    agent:
    {
        type: Person
        name: x.name
        additionalName: x.middle_name
    }
""")

In [11]:
print(forge.map(scientists[1], mapping_na))

{
    type: Contribution
    agent:
    {
        type: Person
        additionalName: (missing)
        name: Albert Einstein
    }
}


In [12]:
print(forge.map(scientists[1], mapping_na, na="(missing)"))

{
    type: Contribution
    agent:
    {
        type: Person
        name: Albert Einstein
    }
}


### multiple mappings

In [13]:
mapping_person = DictionaryMapping("""
    id: forge.format("identifier", "persons", x.id)
    type: Person
    name: x.name
""")

In [14]:
mapping_association = DictionaryMapping("""
    type: Contribution
    agent: forge.format("identifier", "persons", x.id)
""")

In [15]:
resources_mapped = forge.map(scientists, [mapping_person, mapping_association])

In [16]:
len(resources_mapped) == 4

True

In [17]:
print(*resources_mapped, sep="\n")

{
    id: https://kg.example.ch/persons/123
    type: Person
    name: Marie Curie
}
{
    type: Contribution
    agent: https://kg.example.ch/persons/123
}
{
    id: https://kg.example.ch/persons/456
    type: Person
    name: Albert Einstein
}
{
    type: Contribution
    agent: https://kg.example.ch/persons/456
}


## Managing mappings

In [18]:
filepath = "mappings/scientists-database/DictionaryMapping/Contribution.hjson"

### loading

In [19]:
loaded = DictionaryMapping.load(filepath)

### saving

In [20]:
loaded.save(filepath)